<a href="https://colab.research.google.com/github/luasm17/LLM_as_a_judge/blob/main/prueba_1_selene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar dependencias
!pip install -q transformers accelerate

# Imports
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata

In [ ]:
HF_TOKEN = userdata.get("HF_TOKEN")

assert HF_TOKEN is not None, "Non se atopou HF_TOKEN nos secrets de Colab"

In [ ]:
# Selene-1-Mini-Llama-3.1-8B
# LLM-as-a-Judge binario para concordancia de número en galego

# Cargar modelo e tokenizer
MODEL_ID = "AtlaAI/Selene-1-Mini-Llama-3.1-8B"

print("🔄 Cargando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    token=HF_TOKEN
)

print("🔄 Cargando modelo (pode tardar uns minutos)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    token=HF_TOKEN,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

model.eval()
print("✅ Modelo cargado correctamente")

In [ ]:
# Función LLM-as-a-Judge (concordancia de número)
def selene_judge_concordancia(oracion: str) -> str:
    prompt = f"""
Es un LLM-as-a-judge que vai evaluar un modelo de corrección gramatical en galego.

A túa tarefa é avaliar a saída do modelo de corrección gramatical (GEC)
e decidir se a corrección é ADECUADA con respecto á concordancia de número
(singular/plural entre determinante, substantivo, adxectivo ou verbo).

Devolve a resposta EXACTAMENTE co seguinte formato, sen texto adicional:

output_modelo: "<oración avaliada>"
etiqueta: <0 ou 1>
explicacion: "<explicación breve e precisa en galego>"

Criterios:
- etiqueta = 1 → a corrección NON é adecuada (segue habendo erro de concordancia de número)
- etiqueta = 0 → a corrección é adecuada (non hai erro de concordancia de número)
- A explicación debe xustificar só a concordancia de número

NON DEBES, BAIXO NINGÚN CONCEPTO, CORRIXIR O OUTPUT DO MODELO QUE TES QUE AVALIAR. TES QUE LIMITARTE EXCLUSIVAMENTE A DECIDIR SE CONTÉN ERRO DE CONCORDANCIA OU NON.
NON DEBES AVALIAR OUTROS TIPOS DE ERROS.

Agora avalía a seguinte saída dun modelo de GEC:

"<OUTPUT_MODELO>"


"{oracion}"
"""

    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    resposta = tokenizer.decode(
        output[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    return resposta.strip()

In [ ]:
# Probas cos meus exemplos
exemplos = [
    "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas.",
    "O grupo de estudantes que participaron no proxecto presentou os resultados finais onte pola tarde.",
    "As propostas que chegaron desde os concellos máis pequenos foi analizada polo equipo técnico.",
    "A maioría das persoas entrevistadas manifestaron a súa opinión durante a sesión pública."
]

for i, frase in enumerate(exemplos, 1):
    print(f"\n===== EXEMPLO {i} =====")
    print(selene_judge_concordancia(frase))


===== EXEMPLO 1 =====
output_modelo: "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas."
etiqueta: 0
explicacion: A concordancia de número é correcta, pois o verbo "foron" concorda con o substantivo plural "decisións".

===== EXEMPLO 2 =====
output_modelo: "O grupo de estudantes que participaron no proxecto presentou os resultados finais onte pola tarde."
etiqueta: 1
explicación: O verbo "presentou" debe ser "presentaron" para concordar con o suxeito plural "estudantes".

===== EXEMPLO 3 =====
output_modelo: "As propostas que chegaron desde os concellos máis pequenos foi analizada polo equipo técnico."
etiqueta: 1
explicación: O verbo "foi" debe ser "foron" para concordar co plural do substantivo "propostas".

===== EXEMPLO 4 =====
output_modelo: "A maioría das persoas entrevistadas manifestaron a súa opinión durante a sesión pública."
etiqueta: 1
explicación: O verbo "manifestaron" non concorda en número coa maioría, que é plural, xa que debería